In [0]:
# Import Spark SQL and Spark ML libraries
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.mllib.tree import RandomForest
from pyspark.ml import Pipeline


from pyspark.ml.classification import DecisionTreeClassifier

from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler

from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier



In [0]:
IS_SPARK_SUBMIT_CLI = False
if IS_SPARK_SUBMIT_CLI:
    sc = SparkContext.getOrCreate()
    spark = SparkSession(sc)

In [0]:
if IS_SPARK_SUBMIT_CLI:
    coviddeath = spark.read.csv('UScasestemp1.csv', inferSchema=True, header=True)
else:
    coviddeath = spark.sql("SELECT * FROM uscasestemp1_csv")

In [0]:
data = coviddeath.select("Year","Date","Day", "Temp","Lat","Long","Admin2","Province",((col("Case") > 2).cast("Double").alias("label")))
data = StringIndexer(inputCol='Admin2', outputCol='Admin2'+"_index").fit(data).transform(data)
data = StringIndexer(inputCol='Province', outputCol='Province'+"_index").fit(data).transform(data)
data.show(5)


+----+----+---+----+-----------+------------+-----------+----------+-----+------------+--------------+
Year|Date|Day|Temp| Lat| Long| Admin2| Province|label|Admin2_index|Province_index|
+----+----+---+----+-----------+------------+-----------+----------+-----+------------+--------------+
2020| 22| 1| 61|34.30828379|-118.2282411|Los Angeles|California| 0.0| 0.0| 0.0|
2020| 23| 1| 59|34.30828379|-118.2282411|Los Angeles|California| 0.0| 0.0| 0.0|
2020| 24| 1| 63|34.30828379|-118.2282411|Los Angeles|California| 0.0| 0.0| 0.0|
2020| 25| 1| 64|34.30828379|-118.2282411|Los Angeles|California| 0.0| 0.0| 0.0|
2020| 26| 1| 58|34.30828379|-118.2282411|Los Angeles|California| 0.0| 0.0| 0.0|
+----+----+---+----+-----------+------------+-----------+----------+-----+------------+--------------+
only showing top 5 rows

In [0]:
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")
train_rows = train.count()
test_rows = test.count()
print ("Training Rows:", train_rows, " Testing Rows:", test_rows)


Training Rows: 309 Testing Rows: 127

In [0]:
from pyspark.ml.classification import RandomForestClassifier
assembler = VectorAssembler(inputCols =["Day","Lat","Province_index","Admin2_index"],outputCol="normfeatures")
minMax = MinMaxScaler(inputCol = assembler.getOutputCol(), outputCol="nfeatures")
featVect = VectorAssembler(inputCols=["nfeatures"], outputCol="features")
lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3)
pipeline = Pipeline(stages=[assembler,minMax,featVect,lr])

In [0]:
piplineModel = pipeline.fit(train)
print("Pipeline complete!")


Pipeline complete!

In [0]:
# piplineModel with train data set applies test data set and generate predictions
prediction = piplineModel.transform(test)
predicted = prediction.select("features", "prediction", "trueLabel")
predicted.show(100, truncate=False)

+----------------------------------------------------------------+----------+---------+
features |prediction|trueLabel|
+----------------------------------------------------------------+----------+---------+
[0.5,0.9999999999999999,1.0,1.0] |1.0 |0.0 |
(4,[0],[0.5]) |1.0 |0.0 |
(4,[0],[0.75]) |1.0 |1.0 |
(4,[0],[1.0]) |1.0 |1.0 |
[1.0,0.42778986293320975,0.6666666666666666,0.6666666666666666] |1.0 |1.0 |
[0.25,0.9999999999999999,1.0,1.0] |0.0 |0.0 |
[0.25,0.42778986293320975,0.6666666666666666,0.6666666666666666]|0.0 |0.0 |
[0.25,0.4851695873118932,0.3333333333333333,0.3333333333333333] |0.0 |0.0 |
[0.5,0.42778986293320975,0.6666666666666666,0.6666666666666666] |1.0 |0.0 |
(4,[0],[0.5]) |1.0 |0.0 |
[0.75,0.4851695873118932,0.3333333333333333,0.3333333333333333] |1.0 |1.0 |
(4,[0],[1.0]) |1.0 |1.0 |
[1.0,0.4851695873118932,0.3333333333333333,0.3333333333333333] |1.0 |1.0 |
[1.0,0.9999999999999999,1.0,1.0] |1.0 |1.0 |
[0.25,0.42778986293320975,0.6666666666666666,0.6666666666666666]|0.0 |0.0 |
[0.5,0.42778986293320975,0.6666666666666666,0.6666666666666666] |1.0 |0.0 |
[0.75,0.4851695873118932,0.3333333333333333,0.3333333333333333] |1.0 |1.0 |
[0.25,0.42778986293320975,0.6666666666666666,0.6666666666666666]|0.0 |0.0 |
(4,[0],[0.25]) |0.0 |0.0 |
[0.5,0.4851695873118932,0.3333333333333333,0.3333333333333333] |1.0 |1.0 |
[0.5,0.42778986293320975,0.6666666666666666,0.6666666666666666] |1.0 |0.0 |
[0.75,0.4851695873118932,0.3333333333333333,0.3333333333333333] |1.0 |1.0 |
(4,[0],[0.25]) |0.0 |0.0 |
[0.5,0.4851695873118932,0.3333333333333333,0.3333333333333333] |1.0 |1.0 |
[0.25,0.4851695873118932,0.3333333333333333,0.3333333333333333] |0.0 |0.0 |
[0.75,0.4851695873118932,0.3333333333333333,0.3333333333333333] |1.0 |1.0 |
[0.5,0.9999999999999999,1.0,1.0] |1.0 |0.0 |
[0.5,0.4851695873118932,0.3333333333333333,0.3333333333333333] |1.0 |1.0 |
[1.0,0.4851695873118932,0.3333333333333333,0.3333333333333333] |1.0 |1.0 |
(4,[0],[1.0]) |1.0 |1.0 |
[0.25,0.4851695873118932,0.3333333333333333,0.3333333333333333] |0.0 |0.0 |
[0.25,0.42778986293320975,0.6666666666666666,0.6666666666666666]|0.0 |0.0 |
[0.5,0.4851695873118932,0.3333333333333333,0.3333333333333333] |1.0 |1.0 |
[0.5,0.9999999999999999,1.0,1.0] |1.0 |0.0 |
[0.75,0.42778986293320975,0.6666666666666666,0.6666666666666666]|1.0 |1.0 |
[0.25,0.42778986293320975,0.6666666666666666,0.6666666666666666]|0.0 |0.0 |
[0.25,0.9999999999999999,1.0,1.0] |0.0 |0.0 |
(4,[0],[0.25]) |0.0 |0.0 |
[0.5,0.9999999999999999,1.0,1.0] |1.0 |0.0 |
[0.5,0.42778986293320975,0.6666666666666666,0.6666666666666666] |1.0 |0.0 |
[0.75,0.4851695873118932,0.3333333333333333,0.3333333333333333] |1.0 |1.0 |
[0.25,0.4851695873118932,0.3333333333333333,0.3333333333333333] |0.0 |0.0 |
[0.25,0.42778986293320975,0.6666666666666666,0.6666666666666666]|0.0 |0.0 |
(4,[0],[0.75]) |1.0 |1.0 |
[0.25,0.4851695873118932,0.3333333333333333,0.3333333333333333] |0.0 |0.0 |
[0.25,0.9999999999999999,1.0,1.0] |0.0 |0.0 |
[0.5,0.9999999999999999,1.0,1.0] |1.0 |1.0 |
(4,[0],[0.75]) |1.0 |1.0 |
[0.25,0.4851695873118932,0.3333333333333333,0.3333333333333333] |0.0 |0.0 |
[0.25,0.9999999999999999,1.0,1.0] |0.0 |0.0 |
(4,[0],[0.75]) |1.0 |1.0 |
(4,[0],[0.25]) |0.0 |0.0 |
[0.5,0.4851695873118932,0.3333333333333333,0.3333333333333333] |1.0 |1.0 |
[0.5,0.42778986293320975,0.6666666666666666,0.6666666666666666] |1.0 |1.0 |
[0.25,0.9999999999999999,1.0,1.0] |0.0 |0.0 |
[0.25,0.4851695873118932,0.3333333333333333,0.3333333333333333] |0.0 |0.0 |
(4,[0],[0.25]) |0.0 |0.0 |
(4,[0],[0.75]) |1.0 |1.0 |
[0.75,0.9999999999999999,1.0,1.0] |1.0 |1.0 |
[0.25,0.4851695873118932,0.3333333333333333,0.3333333333333333] |0.0 |0.0 |
(4,[0],[0.25]) |0.0 |0.0 |
(4,[0],[0.75]) |1.0 |1.0 |
[0.75,0.4851695873118932,0.3333333333333333,0.3333333333333333] |1.0 |1.0 |
[0.25,0.42778986293320975,0.6666666666666666,0.6666666666666666]|0.0 |0.0 |
(4,[0],[0.25]) |0.0 |0.0 |
[0.5,0.4851695873118932,0.3333333333333333,0.3333333333333333] |1.0 |1.0 |
[0.5,0.42778986293320975,0.6666666666

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluation = MulticlassClassificationEvaluator(
    labelCol="trueLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluation.evaluate(prediction)
print("Accuracy of Logistic Regression is: ",accuracy)

Accuracy of Logistic Regression is: 0.9212598425196851

In [0]:
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.0839695